In [3]:
import numpy as np
import pandas as pd
import keras
import os
print(keras.__version__)

2.2.2


In [2]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image

In [4]:
image_size = 224
num_class = len([x for x in os.listdir('train')])
batch_size = 32
num_channels = 3
train_data_dir = 'train'
validation_data_dir = 'test'

In [5]:
num_class

7

# Transfer learning - Bottleneck

In [9]:
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import preprocess_input, decode_predictions

In [7]:
tr_model=ResNet50(include_top=False,
                  weights='imagenet',
                 input_shape = (224, 224, 3),
                 ) #
#tr_model.summary()

C:\Users\student\Anaconda3\envs\tensorflow\lib\site-packages\keras_applications\resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [10]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [11]:
# training set
train_generator = datagen.flow_from_directory(train_data_dir,
                                                    target_size=(image_size,image_size),
                                                    class_mode=None,
                                                    batch_size=batch_size,
                                                    shuffle=False)

bottleneck_features_train = tr_model.predict_generator(train_generator)
train_labels = to_categorical(train_generator.classes , num_classes=num_class)

Found 1083 images belonging to 7 classes.


In [12]:
# Validation set
validation_generator = datagen.flow_from_directory(validation_data_dir, 
                                                        target_size=(image_size,image_size),
                                                        class_mode=None,
                                                        batch_size=batch_size,
                                                       shuffle=False)

bottleneck_features_validation = tr_model.predict_generator(validation_generator)
validation_labels = to_categorical(validation_generator.classes, num_classes=num_class)

Found 538 images belonging to 7 classes.


In [13]:
print(bottleneck_features_train.shape)
print(bottleneck_features_validation.shape)

(1083, 7, 7, 2048)
(538, 7, 7, 2048)


# Model Creation

In [38]:
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization, regularizers
model = Sequential()
model.add(Flatten(input_shape=bottleneck_features_train.shape[1:]))
model.add(Dense(1024, activation='relu'))

#model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(num_class, activation='softmax', name='pred'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_7 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              102761472 
_________________________________________________________________
dropout_6 (Dropout)          (None, 1024)              0         
_________________________________________________________________
pred (Dense)                 (None, 7)                 7175      
Total params: 102,768,647
Trainable params: 102,768,647
Non-trainable params: 0
_________________________________________________________________


In [39]:
# Early stop
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=15, verbose=1, mode='auto')

In [40]:
history = model.fit(bottleneck_features_train, train_labels,
          epochs=50,
          batch_size=batch_size,
          validation_data=(bottleneck_features_validation, validation_labels),
                   callbacks = [early])

Train on 1083 samples, validate on 538 samples
Epoch 1/50
 512/1083 [=============>................] - ETA: 26s - loss: 11.2863 - acc: 0.2559

KeyboardInterrupt: 